In [4]:
# Import msuc21 and set environment variables
from music21 import *
import os
import subprocess
import csv

In [6]:
file = r"C:\Users\InOarby\Documents\MuseScore4\Scores\solo guitar\Minuet by G.P.Telemann.musicxml"
seg_dir = "segment_dir"
mp3_dir = "mp3_dir"
svg_dir = "svg_dir"

In [22]:
# Read the file and show
s = converter.parse(file)
s.show('musicxml')

In [28]:
def divide_score_at_rehearsal_marks(file, output_dir):
    # Load the score
    score = converter.parse(file)
    # Only use the first part
    part = score.parts[0]

    score_name = score.metadata.title + " by " + score.metadata.composer
    score_name = score_name.replace(" ", "_")
    score_name = score_name.replace(".", "_")

    clef = part.measure(1).clef
    key = part.measure(1).keySignature

    #score = score.parts[0]
    # Initial setup
    measures = part.getElementsByClass('Measure')
    split_scores = []
    current_score = None

    # Iterate over the measures
    for measure in measures:
        # Check if the measure has a rehearsal mark
        if any(isinstance(el, expressions.RehearsalMark) for el in measure.elements):
            # If a score is already in progress, add it to the list of split scores
            if current_score is not None and len(current_score.getElementsByClass('Measure')) > 1:
                split_scores.append(current_score)
            # Start a new score with the global information of the original score
            measure.removeByClass(bar.Repeat)
            current_score = part.cloneEmpty()
            current_score.insert(0, measure)
            current_score.insert(0, clef)
            current_score.insert(0, key)
        else:
            # If there is a current score, add the measure to it
            if current_score is not None:
                measure.removeByClass(bar.Repeat)
                current_score.insert(measure.offset, measure)

    # Add the last score if it exists
    if current_score is not None and len(current_score.getElementsByClass('Measure')) > 1:
        split_scores.append(current_score.getElementsByClass('Measure'))

    # Save the split scores to separate MusicXML files
    for i, split_score in enumerate(split_scores):
        split_score.write('musicxml', os.path.join(output_dir, f'{score_name}_{i+1}.musicxml'))


# Call the function
divide_score_at_rehearsal_marks(file, seg_dir)


In [29]:
def convert_musicxml_to_svg(input_dir, output_dir):
    musescore = r'C:\Program Files\MuseScore 4\bin\MuseScore4.exe'
    # Get a list of all MusicXML files in the input directory
    input_files = [f for f in os.listdir(input_dir) if f.endswith('.musicxml')]
    
    for input_file in input_files:
        # Define the input and output paths
        input_path = os.path.join(input_dir, input_file)
        output_path = os.path.join(output_dir, f'{os.path.splitext(input_file)[0]}.svg')
        
        # Convert the MusicXML file to PNG using MuseScore4 command-line interface
        subprocess.run([musescore, '-o', output_path, input_path, '-T', '100'])

# Call the function
convert_musicxml_to_svg(seg_dir, svg_dir)

In [30]:

def convert_musicxml_to_mp3(input_dir, output_dir):
    musescore = r'C:\Program Files\MuseScore 4\bin\MuseScore4.exe'
    # Get a list of all MusicXML files in the input directory
    input_files = [f for f in os.listdir(input_dir) if f.endswith('.musicxml')]
    
    for input_file in input_files:
        # Define the input and output paths
        input_path = os.path.join(input_dir, input_file)
        output_path = os.path.join(output_dir, f'{os.path.splitext(input_file)[0]}.mp3')
        
        # Convert the MusicXML file to PNG using MuseScore4 command-line interface
        subprocess.run([musescore, '-o', output_path, input_path])

# Call the function
convert_musicxml_to_mp3(seg_dir, mp3_dir)

In [61]:
def write_anki_import_file(media_list, output_dir):
    with open(os.path.join(output_dir, 'anki_import.txt'), 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        for m in media_list:  # For each set of MusicXML, PNG, and MP3 files
            front = f'<img src="{m}.svg">'
            back = f'[sound:{m}.mp3]'
            writer.writerow([front, back, f'{m}'])  # The third column is the card tag

media_list = [os.path.splitext(f)[0] for f in os.listdir(seg_dir)]
write_anki_import_file(media_list, './')